In [1]:
from OCC.Core.BRep import BRep_Builder
from OCC.Core.TopoDS import TopoDS_Shape
from OCC.Core.BRepTools import breptools_Read
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.TopoDS import TopoDS_Face
from OCC.Core.BRep import BRep_Tool
from OCC.Core.TopLoc import TopLoc_Location
import pyvista as pv
import numpy as np

# pv.set_jupyter_backend("static")

def load_brep(filepath):
    shape = TopoDS_Shape()
    builder = BRep_Builder()

    success = breptools_Read(shape, filepath, builder)
    if not success:
        raise RuntimeError(f"Failed to load BREP file: {filepath}")

    mesh = BRepMesh_IncrementalMesh(shape, 0.1)
    mesh.Perform()
    return shape

# 3. Face → PyVista PolyData


def extract_faces_to_pyvista(shape):
    explorer = TopExp_Explorer(shape, TopAbs_FACE)
    faces_polydata = []

    while explorer.More():
        shape = explorer.Current()
        face = TopoDS_Face()
        face.TShape(shape.TShape())
        face.Location(shape.Location())
        face.Orientation(shape.Orientation())

        location = TopLoc_Location()
        triangulation = BRep_Tool.Triangulation(face, location)

        if triangulation:
            nb_nodes = triangulation.NbNodes()
            points = np.array([
                [triangulation.Node(i).X(), triangulation.Node(i).Y(), triangulation.Node(i).Z()]
                for i in range(1, nb_nodes + 1)
            ])

            nb_tris = triangulation.NbTriangles()
            faces = []
            for i in range(1, nb_tris + 1):
                tri = triangulation.Triangle(i)
                idx = [tri.Value(j) - 1 for j in range(1, 4)]
                faces.append([3, *idx])

            # PyVista 형상 생성
            polydata = pv.PolyData(points, np.array(faces, dtype=np.int32))
            faces_polydata.append(polydata)

        explorer.Next()

    return faces_polydata

# 4. PyVista 시각화


def show_pyvista_meshes(polydata_list):
    plotter = pv.Plotter()
    for pd in polydata_list:
        plotter.add_mesh(pd, color="lightgray", show_edges=True)
    plotter.show()

In [2]:
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE
# 사용 예
shape = load_brep("../../data/fixed.brep")

# 검증: face 개수 세기
explorer = TopExp_Explorer(shape, TopAbs_FACE)
count = 0
while explorer.More():
    count += 1
    explorer.Next()
print(f"Face count: {count}")

Face count: 2476


/var/folders/2n/sygb5rn93wb8t06pd18v3yxw0000gn/T/ipykernel_14757/2957943816.py:19: DeprecationWarning: Call to deprecated function breptools_Read since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method breptools.Read
  success = breptools_Read(shape, filepath, builder)


In [3]:
polydata_list = extract_faces_to_pyvista(shape)
polydata_list

[PolyData (0x3098a6740)
   N Cells:    18
   N Points:   16
   N Strips:   0
   X Bounds:   -1.524e+01, -1.137e+01
   Y Bounds:   -6.296e+01, -6.250e+01
   Z Bounds:   3.803e+01, 4.085e+01
   N Arrays:   0,
 PolyData (0x3098a69e0)
   N Cells:    10
   N Points:   10
   N Strips:   0
   X Bounds:   -2.607e+01, -2.359e+01
   Y Bounds:   -6.288e+01, -6.269e+01
   Z Bounds:   3.770e+01, 3.943e+01
   N Arrays:   0,
 PolyData (0x3098a6c80)
   N Cells:    17
   N Points:   15
   N Strips:   0
   X Bounds:   -2.982e+01, -2.561e+01
   Y Bounds:   -6.364e+01, -6.307e+01
   Z Bounds:   5.152e+01, 5.540e+01
   N Arrays:   0,
 PolyData (0x3098a6da0)
   N Cells:    17
   N Points:   15
   N Strips:   0
   X Bounds:   -2.690e+01, -2.221e+01
   Y Bounds:   -6.357e+01, -6.308e+01
   Z Bounds:   5.437e+01, 5.731e+01
   N Arrays:   0,
 PolyData (0x3098a6f20)
   N Cells:    18
   N Points:   16
   N Strips:   0
   X Bounds:   -1.881e+01, -1.479e+01
   Y Bounds:   -6.329e+01, -6.287e+01
   Z Bounds:   5.49

In [4]:
show_pyvista_meshes(polydata_list)

Widget(value='<iframe src="http://localhost:51670/index.html?ui=P_0x313a85010_0&reconnect=auto" class="pyvista…